# Extending Logistic Regression: 2009 American Community Survey

### Nick Chao

### Preparation and Overview (20 points total)

[10 points] Explain the task and what business-case or use-case it is designed to solve (or designed to investigate). Detail exactly what the classification task is and what parties would be interested in the results.

[5 points] (mostly the same processes as from previous labs) Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis. Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

[5 points] Divide you data into training and testing data using an 80% training and 20% testing split. Use the cross validation modules that are part of scikit-learn. Argue "for" or "against" splitting your data using an 80/20 split. That is, why is the 80/20 split appropriate (or not) for your dataset? 


### Modeling (50 points total)

[20 points] Create a custom, one-versus-all logistic regression classifier using numpy and scipy to optimize. Use object oriented conventions identical to scikit-learn. You should start with the template developed by the instructor in the course. You should add the following functionality to the logistic regression classifier:
Ability to choose optimization technique when class is instantiated: either steepest descent, stochastic gradient descent, or Newton's method. 
Update the gradient calculation to include a customizable regularization term (either using no regularization, L1 regularization, L2 regularization, or both L1 and L2 regularization). Associate a cost with the regularization term, "C", that can be adjusted when the class is instantiated.  

[15 points] Train your classifier to achieve good generalization performance. That is, adjust the optimization technique and the value of the regularization term "C" to achieve the best performance on your test set. Visualize the performance of the classifier versus the parameters you investigated. Is your method of selecting parameters justified? That is, do you think there is any "data snooping" involved with this method of selecting parameters?

[15 points] Compare the performance of your "best" logistic regression optimization procedure to the procedure used in scikit-learn. Visualize the performance differences in terms of training time and classification performance. Discuss the results. 

### Deployment (10 points total)
Which implementation of logistic regression would you advise be used in a deployed machine learning model, your implementation or scikit-learn (or other third party)? Why?

### Exceptional Work (10 points total)
You have free reign to provide additional analyses. One idea: Update the code to use either "one-versus-all" or "one-versus-one" extensions of binary to multi-class classification. 
One idea (required for 7000 level students): Implement an optimization technique for logistic regression using mean square error as your objective function (instead of binary entropy). Your solution should be able to solve the binary logistic regression problem in one gradient update step. 


## Data Overview

In this lab, we investigate possible relationships between an individual's income and attributes about them. The data used in this lab is provided by the 2009 American Community Survey 1-Year PUMS Population File which can be found here. https://catalog.data.gov/dataset/2009-american-community-survey-1-year-pums-population-file. This dataset contains more than 3 million entries and nearly 300 attributes. To make sense of some of these attributes, there is a dictionary lookup that provides more information about the columns. You can find this reference here. https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMSDataDict09.pdf.

To be more specific, we want to see if we can predict a person's current income based on factors about themselves that they might give away when applying for a new job. For example, their age, sex, education level, and more are just a few peices of information that companies may ask for when applying for a new job. 

This information could be incrediably useful for a company that is hiring new personal. If they know the current income of someone who has applied for a job at their company, then they can offer the lowest starting salary they believe the new canadate will accept. (i.e. slightly above what they currently make). 


In [1]:
#importing dependancies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression

from sklearn import metrics as mt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.utils.estimator_checks import check_estimator
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.metrics import accuracy_score
from scipy.optimize import fmin_bfgs
from scipy.special import expit

In [2]:
%time dataA = pd.read_csv('../data/ss09pusa.csv')
%time dataB = pd.read_csv('../data/ss09pusb.csv')
merged = pd.concat([dataA,dataB])

Wall time: 27.6 s
Wall time: 27.8 s


In [3]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3030728 entries, 0 to 1466654
Columns: 279 entries, RT to pwgtp80
dtypes: float64(86), int64(190), object(3)
memory usage: 6.3+ GB


In [4]:
print('The 2009 American Community Survey has a lot of data, look at all these columns! \n\n'+str(list(merged.columns.values)))
print('\nNo wonder they provide a reference dictionary to figure out what all these acronyms mean')

The 2009 American Community Survey has a lot of data, look at all these columns! 

['RT', 'SERIALNO', 'SPORDER', 'PUMA', 'ST', 'ADJINC', 'PWGTP', 'AGEP', 'CIT', 'CITWP', 'COW', 'DDRS', 'DEAR', 'DEYE', 'DOUT', 'DPHY', 'DRAT', 'DRATX', 'DREM', 'ENG', 'FER', 'GCL', 'GCM', 'GCR', 'HINS1', 'HINS2', 'HINS3', 'HINS4', 'HINS5', 'HINS6', 'HINS7', 'INTP', 'JWMNP', 'JWRIP', 'JWTR', 'LANX', 'MAR', 'MARHD', 'MARHM', 'MARHT', 'MARHW', 'MARHYP', 'MIG', 'MIL', 'MLPA', 'MLPB', 'MLPC', 'MLPD', 'MLPE', 'MLPF', 'MLPG', 'MLPH', 'MLPI', 'MLPJ', 'MLPK', 'NWAB', 'NWAV', 'NWLA', 'NWLK', 'NWRE', 'OIP', 'PAP', 'REL', 'RETP', 'SCH', 'SCHG', 'SCHL', 'SEMP', 'SEX', 'SSIP', 'SSP', 'WAGP', 'WKHP', 'WKL', 'WKW', 'WRK', 'YOEP', 'ANC', 'ANC1P', 'ANC2P', 'DECADE', 'DIS', 'DRIVESP', 'ESP', 'ESR', 'FOD1P', 'FOD2P', 'HICOV', 'HISP', 'INDP', 'JWAP', 'JWDP', 'LANP', 'MIGPUMA', 'MIGSP', 'MSP', 'NAICSP', 'NATIVITY', 'NOP', 'OC', 'OCCP', 'PAOC', 'PERNP', 'PINCP', 'POBP', 'POVPIP', 'POWPUMA', 'POWSP', 'PRIVCOV', 'PUBCOV', 'QTRBIR

### Data Preparation

Obviously there is way more data here than we need so lets start by reducing the number of columns to only what we consider useful for our classification. The following attributes will remain as they are peices of information that new hires might give away when applying for a job.

Citizenship, age, class of work, English fluentcy, martial status, military status, sex, education background, disability status, race, geographical location. We will also keep the individual's income as it is the attribute that we are attempting to predict. 

In [5]:
cols_to_save = ['CIT','AGEP','COW','ENG','MAR','MIL','SCHL','SEX','DIS','PINCP','POWSP','RAC1P','FOD1P']
new_data = merged.filter(items=cols_to_save)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3030728 entries, 0 to 1466654
Data columns (total 13 columns):
CIT      int64
AGEP     int64
COW      float64
ENG      float64
MAR      int64
MIL      float64
SCHL     float64
SEX      int64
DIS      int64
PINCP    float64
POWSP    float64
RAC1P    int64
FOD1P    float64
dtypes: float64(7), int64(6)
memory usage: 323.7 MB


In [6]:
new_data

,CIT,AGEP,COW,ENG,MAR,MIL,SCHL,SEX,DIS,PINCP,POWSP,RAC1P,FOD1P
0,1,51,NaN,NaN,4,3.0,16.0,1,1,3800.0,NaN,1,NaN
1,1,64,1.0,NaN,1,5.0,19.0,2,1,36800.0,1.0,1,NaN
2,1,68,1.0,NaN,1,3.0,14.0,1,1,54600.0,1.0,1,NaN
3,1,61,1.0,NaN,3,5.0,16.0,2,2,6000.0,1.0,2,NaN
4,1,38,1.0,NaN,5,5.0,16.0,1,2,14000.0,1.0,2,NaN
5,1,65,6.0,NaN,1,5.0,16.0,2,1,13000.0,NaN,1,NaN
6,1,74,1.0,NaN,1,3.0,16.0,1,1,45200.0,1.0,1,NaN
7,1,23,1.0,NaN,5,5.0,12.0,2,1,820.0,NaN,1,NaN
8,1,42,1.0,NaN,1,5.0,19.0,2,2,25200.0,1.0,1,NaN
9,1,42,1.0,NaN,1,5.0,13.0,1,2,56000.0,1.0,1,NaN


We began with over 3 million records and 279 differnt attributes provided in the survey. Of the nearly 300 attributes that were provided in the dataset, we reduced it down to the thirteen shown above. The table below shows the desired attributes and examples of what we would want the data to look like. 

|Attribute|Description|Type|Example|
|:---:|:---:|:---:|:---:|
| CIT | Citizenship Status | Int | 1. Citizen, 0. Non-citizen |
| AGEP | Age | Int | 23
| COW | Class of Worker | Float | 3. Local Government, 4. State Government |
| ENG | Ability to speak English  | Int | 1. Speaks English, 0. Doesn't Speak English |
| MAR | Marital Status | Int | 1. Married, 2. Widowed |
| MIL | Military Service | Int | 1. Yes, 0. No |
| SCHL | Educational Attainment  | Float | 21 Bachelor's Degree, 22 Master's Degree |
| SEX | Sex      | Int | True. Male |
| DIS | Disability | Int | True. Disabled |
| PINCP | Total Person's Income | Float
| POWSP | Place of work | Float | 048 Texas, 049 Utah |
| RAC1P | Detailed Race Code | Int | 1 White, 6 Asian |
| FOD1P | Field of Degree | Float | 2407 Computer Engineering, 2408 Electrical Engineering |

### Data Cleaning

The next step is data cleaning and modifying the data into more useful data types. Lets start by modifying some of these data types into more useful and proper variables.

In [7]:
# Change citizenship to Int.
# 1-4 is a citizen (true) and 5 is not a citizen (false)

new_data.CIT.replace(to_replace = range(5),
                    value=[1,1,1,1,0],
                    inplace=True)
new_data['CIT'] = new_data['CIT'].astype('int')

In [8]:
# Change Ability to Speak English to boolean
# b is N/A but it would be a good assumption to assume they speak English
new_data['ENG']=new_data['ENG'].fillna(1)
# 1-2 speaks English well or very well, 3-4 speaks English not well or not at all.
new_data.ENG.replace(to_replace = range(4),
                    value=[1,1,0,0],
                    inplace=True)
new_data['ENG'] = new_data['ENG'].astype('int')# Change Military Status to Boolean

In [9]:
# b is N/A because less than 17 years old so lets just change this to 0
new_data['MIL']=new_data['MIL'].fillna(0)
# 1-3 Yes, 4-5 No
new_data.MIL.replace(to_replace = range(5),
                    value=[1,1,1,0,0],
                    inplace=True)
new_data['MIL'] = new_data['MIL'].astype('int')

In [10]:
# # Change Sex to Int
# # 1 is male, 2 is female. Changing 2 to 0 for boolean conversion
# new_data.SEX.replace(to_replace = range(2),
#                     value=[1,0],
#                     inplace=True)
# new_data['SEX'] = new_data['SEX'].astype('Int')

In [11]:
# # Change DIS to Int
# # 1 is disabled, 2 is no disability. Changing 2 to 0 for boolean conversion
# new_data.DIS.replace(to_replace = range(2),
#                     value=[1,0],
#                     inplace=True)
# new_data['DIS'] = new_data['DIS'].astype('Int')

In [12]:
# Change Educational Atttainment to INT
# bb is N/A for less than 3 years old.
new_data['SCHL']=new_data['SCHL'].fillna(0)
# For this classification lets simplify some of these education levels.
# 0 between No schooling and Grade 8
# 1 between Grade 9 and Grade 12 no diploma
# 2 for High School degree or GED
# 3 Some college to Associate's degree
# 4 Bachelor's Degree
# 5 Master's Degree
# 6 Professional degree or Doctorate
new_data.SCHL.replace(to_replace = range(25),
                    value=[0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,2,2,3,3,3,4,5,6,6],
                    inplace=True)
new_data['SCHL'] = new_data['SCHL'].astype('int')

Next, lets remove all entries with people under the age of 18 because our goal is to focus on personal income of working class individuals.

In [13]:
# delete younger than 18
new_data = new_data[new_data.AGEP >= 18]
#new_data

Now lets work on the Nulls...

In [14]:
# find null columns
print('Columns that contain nulls: '+str(new_data.columns[new_data.isnull().any()].tolist()))

Columns that contain nulls: ['COW', 'POWSP', 'FOD1P']


For Field of Study, we can replace all the Nulls with 0s since they only refer to those with at least a college degree
Let's remove any Class of Worker and Place of Work rows with Nulls since those entres are for idividuals who have not worked

In [15]:
# Field of Study  -> 0
# Class of Worker -> Remove if Null
# Place of Work   -> Remove if Null

new_data['FOD1P'].fillna(0, inplace=True)
new_data = new_data[pd.notnull(new_data['COW'])]
new_data = new_data[pd.notnull(new_data['POWSP'])]

# Convert the Floats to Ints
# COW, POWSP, FOD1P, PINCP

new_data['COW'] = new_data['COW'].astype('int')
new_data['POWSP'] = new_data['POWSP'].astype('int')
new_data['FOD1P'] = new_data['FOD1P'].astype('int')
new_data['PINCP'] = new_data['PINCP'].astype('int')

C:\Users\Nick\Anaconda3\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [16]:
# Double check for null columns
print('Columns that contain nulls: '+str(new_data.columns[new_data.isnull().any()].tolist()))

Columns that contain nulls: []


We will start by splitting the personal income data into 2 different classes to test Binary Logistic Regression. Later, we will split the personal income into 5 different classes. This will help verify our income predictions while also being specific enough to work with our business goal.

In [17]:
future_data = new_data.copy(deep=False) # saving a copy for later

new_data['PINCP'] = pd.qcut(new_data.PINCP, 2, labels=[0,1])
new_data['PINCP'].unique()

#qcut(x, q, labels=None, retbins=False, precision=3, duplicates='raise')
#cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False)

#new_data['PINCP'] = pd.qcut(new_data.PINCP, 5, labels=[0,1,2,3,4])
#new_data['PINCP'].unique()

[1, 0]
Categories (2, int64): [0 < 1]

In [18]:
new_data['PINCP'] = new_data['PINCP'].astype(np.int)

In [19]:
# Lets see how the income classes have split...
print('Number of people in each class:')
for value in new_data.PINCP.unique(): 
    print(str(value)+': ' +str(len(new_data[new_data['PINCP'] == value])))

Number of people in each class:
1: 660205
0: 678074


In [20]:
# Finally, let's rename some of these columns so they make more sense.
new_data.rename(columns={'CIT': 'Citizenship','AGEP': 'Age','COW': 'Class of Work','ENG': 'Speaks English','MAR': 'Martial Status','MIL': 'Military Status','SCHL': 'Education Level','SEX': 'Male','DIS': 'Disabled?','PINCP': 'Income','POWSP': 'Place of Work','RAC1P': 'Race','FOD1P': 'Field of Study'}, inplace=True)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338279 entries, 1 to 1466654
Data columns (total 13 columns):
Citizenship        1338279 non-null int32
Age                1338279 non-null int64
Class of Work      1338279 non-null int32
Speaks English     1338279 non-null int32
Martial Status     1338279 non-null int64
Military Status    1338279 non-null int32
Education Level    1338279 non-null int32
Male               1338279 non-null int64
Disabled?          1338279 non-null int64
Income             1338279 non-null int32
Place of Work      1338279 non-null int32
Race               1338279 non-null int64
Field of Study     1338279 non-null int32
dtypes: int32(8), int64(5)
memory usage: 102.1 MB


### Training and Testing

As we can see above, there is still a significant amount of data for testing and training purposes (Over 1.3 million entries). The next step is to determine how much of the data should be used immediately for training purposes and how much should be saved until the end for testing purposes. 

An 80/20 split seems to be a good number for this situation. As long as the split is randomized and there would be over a million entries just for training, and around a quarter of a million entries for testing. That seems like plenty of data to create a accurate model as well as test the model for it's accuracy. Furthermore, since we will be spliting the income into five classes, the 80/20 split seems appropriate.

In [21]:
# Lets start by making a copy of the cleaned data we are using.
new_df = new_data.copy()
if 'Income' in new_data:
    y = new_df['Income'].values    # Since Income is our target class, lets make a copy of it
    del new_df['Income']           # Now we need to remove the target class
    X = new_df.values              # The remaining data will be used to train

# Scikit Learns provides a way to split our data into training and testing subsets.
cv_object = ShuffleSplit(train_size=.8, test_size=0.2, n_splits=1)
                         
print(cv_object)

ShuffleSplit(n_splits=1, random_state=None, test_size=0.2, train_size=0.8)


## Modeling

The goal is to create a custom, one-versus-all logistic regression classifier using numpy and scipy to optimize. 
Let's start by using the template from: https://github.com/eclarson/MachineLearningNotebooks/blob/master/05.%20Logistic%20Regression.ipynb

We will add and modify functions as required.

In [22]:
import warnings
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta, iterations=20, c=0.001, norm=0.5):
        self.eta = eta
        self.iters = iterations
        self.c = c
        if((0 <= norm <= 1) or ( norm == -1)): #Check to see if the l1 norm is valid
            self.norm = norm
        else: raise ValueError("Norm must be between 0 and 1 or 0 and -1")
        # internally we will store the weights as self.w_ to keep with sklearn conventions
        
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
    
    # convenience, private:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_bias(x):
        return np.hstack((np.ones((x.shape[0],1)),x)) # add bias term
     
    # public:
    def _normalize(self, gradient):
        #Implementation for Elastic Net regularization 
        sub_1 = np.copy(gradient[1:])
        sub_2 = np.copy(gradient[1:])
        
        #Calculate L1 Norm 
        mask = np.logical_and(sub_1 >= (-self.c/2),sub_1 <= (self.c/2))
        sub_1[mask] = 0
        sub_1[sub_1 < (-self.c/2)] += (self.c / 2)
        sub_1[sub_1 > (self.c/2)] -= (self.c / 2)
        
        #Calculate L2 Norm
        sub_2 += -2 * self.w_[1:] * self.c
        
        #Combine the regularizations to make an elastic net.
        gradient[1:] = self.norm * sub_1 + (1-self.norm) * sub_2
        return gradient
    
    def newtonNormalize(self, w, gradient):
        # regularization (adds both if 3)
        if self.norm & 1: # L1 norm
            gradient[1:] += -1 * w[1:] * self.c
        elif self.norm & 2: # L2 norm
            gradient[1:] += -2 * w[1:] * self.c
            
    def fit(self, x, y):
        Xb = x
        #Xb = self._add_bias(x) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((x.shape[1],1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            #self.normalize(self.w_, gradient)
            self.w_ += gradient*self.eta # multiply by learning rate 
    
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        #Xb = self._add_bias(X) if add_bias else X
        Xb = X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,x):
        return (self.predict_proba(x)>0.5) #return the actual prediction
    
    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        if (self.norm == -1):
            return gradient
        return self._normalize(gradient)
      

In [23]:
# Let's write some classifiers (Steepest Descent, Stochastic Gradient Descent, and Newton's Method)

# Steepest Gradient Descent algorithm
class BinarySteepDescClassifier(BinaryLogisticRegressionBase):
    def _get_gradient(self, x, y):
        ydiff = y-self.predict_proba(x,add_bias=False).ravel()
        gradient = np.mean(x * ydiff[:,np.newaxis], axis=0)
        return gradient.reshape(self.w_.shape)

In [24]:
# Stochastic Gradient Descent Algorithm
class BinaryStochDescClassifier(BinaryLogisticRegressionBase):
    def _get_gradient(self, x, y):
        ydiff = y-self.predict_proba(x,add_bias=False).ravel() # get y difference
        gradient = np.mean(x * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        if (self.norm == -1):
            return gradient
        return self._normalize(gradient)

In [25]:
# Newton's Method (BFGS)
class BinaryNewtonClassifier(BinaryLogisticRegressionBase):
#     def _hessian(self,x,y):
#         g = self.predict_proba(x,add_bias=False).ravel()
#         if (self.norm == -1):
#             return x.T @ np.diag(g*(1-g)) @ x - 2
#         else:
#             return x.T @ np.diag(g*(1-g)) @ x - 2 * (1-self.norm) * self.c
    
#     def fit(self, x, y):
#         Xb = self._add_bias(x)
#         num_samples, num_features = Xb.shape
#         self.w_ = np.zeros((num_features,1))
#         for _ in range(self.iters):
#             gradient = self._get_gradient(Xb,y)
#             inv_hessian = np.linalg.inv(self._hessian(Xb,y))
#             self.w_ +=  inv_hessian@gradient*self.eta 
    def fit(self, x, y):
        def obj_fn(w, x, y, c):
            g = expit(x @ w)
            return -np.sum(np.log(g[y==1]))-np.sum(np.log(1-g[y==0])) + c*sum(w**2)
        
        def obj_grad(w, x, y, c):
            g = expit(x @ w)
            ydiff = y-g
            gradient = np.mean(x * ydiff[:,np.newaxis], axis=0)
            gradient = gradient.reshape(w.shape)
            self.newtonNormalize(w, gradient)
            return -gradient
        
        self.w_ = fmin_bfgs(obj_fn, 
                            np.zeros((x.shape[1], 1)), 
                            fprime=obj_grad, 
                            args=(x, y, self.c), 
                            gtol=1e-03, 
                            maxiter=self.iters,
                            disp=False).reshape((x.shape[1], 1))
        

Lets start by using scikit-learn's logistic refression classifier as a base of reference.

In [26]:
lr_clf = LogisticRegression()

# We will use the cv_object we created earilier and iterate through the different training and testing sets.

iter_num=0
for train_indices, test_indices in cv_object.split(X):
    print("==== Iteration",iter_num,"====")
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    # train the reusable logisitc regression model on the training data
    %time lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    for i in range(0,len(conf)):
        print("Accuracy for income bracket", i, ":",conf[i][i]/sum(conf[i]))
    print("Overall accuracy:", acc )
    print("Confusion matrix\n",conf)
    iter_num += 1

==== Iteration 0 ====
Wall time: 9.5 s
Accuracy for income bracket 0 : 0.7329644129161393
Accuracy for income bracket 1 : 0.7028095686226891
Overall accuracy: 0.7181195265564755
Confusion matrix
 [[99604 36288]
 [39159 92605]]


Our base accuracy is around 72%. Note that this accuracy will vary everytime you run the above code.
Let's now test our Binary classifiers...

In [27]:
%%time
#Let's start with Steepest Descent Classifer
steep = BinarySteepDescClassifier(0.1,1000, c=0.001,norm=1)

steep.fit(X,y)
yhat = steep.predict(X)
print (steep)
print('Accuracy: ' , accuracy_score(y,yhat))

C:\Users\Nick\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp


Binary Logistic Regression Object with coefficients:
[[-3.13174691]
 [ 1.72797173]
 [-1.06074975]
 [-0.27770514]
 [-6.21497795]
 [-5.01860433]
 [ 4.19562217]
 [-5.17155836]
 [-0.47226692]
 [ 0.15690308]
 [-1.96547819]
 [98.72188756]]
Accuracy:  0.6044337540976135
Wall time: 2min 58s


In [28]:
%%time
# Now Let's do Stochastic Descent Classifer
stoch = BinaryStochDescClassifier(0.1,1000, c=0.001,norm=1)

stoch.fit(X,y)
yhat = stoch.predict(X)
print (stoch)
print('Accuracy: ' , accuracy_score(y,yhat))

C:\Users\Nick\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp


Binary Logistic Regression Object with coefficients:
[[-3.16080651]
 [ 1.74469006]
 [-1.04963685]
 [-0.29419391]
 [-6.37202969]
 [-5.07017894]
 [ 4.18532568]
 [-5.18448481]
 [-0.48902674]
 [ 0.26390455]
 [-1.99068996]
 [98.74388974]]
Accuracy:  0.6000968407932875
Wall time: 2min 56s


In [29]:
%%time
# Now Let's do Newton's Method Classifer (BFGS)
newton = BinaryNewtonClassifier(0.1,10, c=0.001,norm=0)

newton.fit(X,y)
yhat = newton.predict(X)
print (newton)
print('Accuracy: ' , accuracy_score(y,yhat))


C:\Users\Nick\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log
C:\Users\Nick\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log


Binary Logistic Regression Object with coefficients:
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Accuracy:  0.5066761116329256
Wall time: 2.4 s


It should be noted that the above cells were run using binary logistic regression. In this dataset it's goal is to determine whether a person is in the top income bracket or the bottom income bracket. Although it is not entirely useful, starting from these binary classifers we can build a custom multi class logistic regression classifier for our intended goal of 5 different income brackets. 

We will also need to optimize these regression classifiers as we can see above they are less efficient than the logistic regression classifier from sklearn. First thing to do is to bring in our old copy of the data so we can divide the income brackets into five classes instead of two.

In [32]:
future_data['PINCP'] = pd.qcut(future_data.PINCP, 5, labels=[0,1,2,3,4])
future_data['PINCP'].unique()

future_data['PINCP'] = future_data['PINCP'].astype(np.int)

ValueError: Bin edges must be unique: array([0., 0., 1., 2., 3., 4.]).
You can drop duplicate edges by setting the 'duplicates' kwarg

In [34]:
# Lets see how the income classes have split...
print('Number of people in each class:')
for value in future_data.PINCP.unique(): 
    print(str(value)+': ' +str(len(future_data[future_data['PINCP'] == value])))

Number of people in each class:
2: 233964
3: 268048
0: 267705
1: 302608
4: 265954


In [35]:
# Finally, let's rename some of these columns so they make more sense.
future_data.rename(columns={'CIT': 'Citizenship','AGEP': 'Age','COW': 'Class of Work','ENG': 'Speaks English','MAR': 'Martial Status','MIL': 'Military Status','SCHL': 'Education Level','SEX': 'Male','DIS': 'Disabled?','PINCP': 'Income','POWSP': 'Place of Work','RAC1P': 'Race','FOD1P': 'Field of Study'}, inplace=True)
future_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338279 entries, 1 to 1466654
Data columns (total 13 columns):
Citizenship        1338279 non-null int32
Age                1338279 non-null int64
Class of Work      1338279 non-null int32
Speaks English     1338279 non-null int32
Martial Status     1338279 non-null int64
Military Status    1338279 non-null int32
Education Level    1338279 non-null int32
Male               1338279 non-null int64
Disabled?          1338279 non-null int64
Income             1338279 non-null int32
Place of Work      1338279 non-null int32
Race               1338279 non-null int64
Field of Study     1338279 non-null int32
dtypes: int32(8), int64(5)
memory usage: 102.1 MB


In [36]:
# Lets start by making a copy of the cleaned data we are using.
future_df = future_data.copy()
if 'Income' in future_data:
    y = future_df['Income'].values    # Since Income is our target class, lets make a copy of it
    del future_df['Income']           # Now we need to remove the target class
    X = future_df.values              # The remaining data will be used to train

# Scikit Learns provides a way to split our data into training and testing subsets.
cv_object = ShuffleSplit(train_size=.8, test_size=0.2, n_splits=1)
                         
print(cv_object)

ShuffleSplit(n_splits=1, random_state=None, test_size=0.2, train_size=0.8)


### Multi Class Logistic Regression